In [1]:
import re
import time
import pickle
import asyncio
import datetime
import requests
import numpy as np
import pandas as pd 
import nest_asyncio
from utils import *
from tqdm import tqdm
from os import listdir
from collections import Counter
from bilibili_api import video, sync

nest_asyncio.apply()

In [2]:
cat_dic = init_category_dic()
tid_lst = np.array(list(cat_dic.keys()))
print('Category list initialized successfully\nWe have {} categories'.format(len(tid_lst)))

Category list initialized successfully
We have 126 categories


In [3]:
for curr_id in tqdm(tid_lst):
    tmp_lst = process_category(curr_id)
    fname = 'pickle/part{}.pkl'.format(curr_id)
    with open(fname, 'wb') as f:
        pickle.dump(tmp_lst, f)

100%|████████████████████████████████████████████████████████████████████████████████| 126/126 [37:36<00:00, 17.91s/it]


In [3]:
all_data = []
for i in listdir('pickle'):
    if 'pkl' in i:
        all_data = all_data + pickle.load(open('pickle/{}'.format(i), 'rb'))

In [4]:
df = list_to_csv(all_data)

In [5]:
df[-20:]

,Bullet Chat,Frequency,BVID,Source Video Title,Category ID,Channel ID,Source Video View Count
868990,好耶,632,BV1BL4y137Mo,咩栗x呜米哈索尔原创曲,28,呜米,132517
711075,很蓝的啦,673,BV1UT4y1X7bT,很蓝的啦,22,超音速君,831813
189320,我能受这委屈我刚学的擒拿术,716,BV1sL411K7mY,想不明白整这么逼真的玩具干啥,138,也就那点事儿-,5304945
61555,新婚快乐,780,BV1Ui4y1173z,结婚就该这么来上才艺吧哈哈哈哈舞台差点干塌,198,OB山下一散人,1590887
1092035,接好运,858,BV1oF41177ui,牛骨天花板,76,澳洲阿彭,1616836
344229,神子不歪玩到关服,858,BV1M341177YQ,原神角色演示八重神子仙狐宫司异闻录,172,原神,3621719
103469,希望手没事看到复制,939,BV1pR4y177WY,我打我自己面对魔哩煞模组来啦周五放克夜FridayNightFunkin魔哩煞,136,魔哩煞,228468
709409,我去初音未来,992,BV1yu41197qg,我去初音未来,22,枪弹轨迹,1420862
859740,致敬,1069,BV1ib4y17793,致黑夜中呜咽与怒吼,27,进击的金厂长,1278026
272336,尺v尺,1164,BV1cq4y1t78f,求这个的出处如何打出,138,瞎问虾猜丶,1264814


# Tokenization

In [6]:
import jieba

In [11]:
bc_lst = df['Bullet Chat'].tolist()
fq_lst = df['Frequency'].tolist()

In [25]:
token_dict = {}
for idx in range(len(bc_lst)):
    seg_list = list(jieba.cut_for_search(bc_lst[idx]))
    for token in seg_list:
        if token in token_dict:
            token_dict[token] = token_dict[token] + fq_lst[idx]
        else:
            token_dict[token] = fq_lst[idx]

In [26]:
token_lst = list(token_dict.keys())
token_count = [token_dict[i] for i in token_lst]

In [27]:
df2 = pd.DataFrame(np.array([token_lst, token_count]).T, columns = ['Word Token', 'Frequency'])

In [28]:
df2

,Word Token,Frequency
0,壮哉,7
1,我,132241
2,大,9247
3,zoom505II,1
4,哇,5297
...,...,...
199182,甜进,41
199183,随烟,43
199184,MeUmyMeUmyMeUmy,79
199185,MeUmyMeUMeUmyMeUmyMeUmyMeUmymyMeUmyMeUmy,121


In [29]:
df2.sort_values(by=['Frequency'])[-20:]

,Word Token,Frequency
62959,黄家驹,99
55042,渴望,99
32669,兔兔,99
26223,giegie,99
79530,无间道,99
7884,魔性,99
11190,开封,99
52478,夏日,99
28462,啪啦,99
911,讲道理,99
